In [25]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import DatasetFolder
from torchsummary import summary
from PIL import Image
from tqdm import tqdm
import os

In [27]:
transform = transforms.Compose(
    [transforms.RandomHorizontalFlip(0.4),
     transforms.RandomVerticalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [37]:
batch_size = 4

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_len = int(len(train_dataset) * 0.8)
val_len = len(train_dataset) - train_len
trainset, valset = torch.utils.data.random_split(train_dataset, [train_len, val_len])

train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

val_loader = DataLoader(valset, batch_size=batch_size, shuffle=True, num_workers=2)

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [31]:
import torch.nn.functional as F
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.batch_nor1 = nn.BatchNorm2d(6)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.batch_nor2 = nn.BatchNorm2d(16)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.batch_nor1(self.conv1(x))))
        x = self.pool(F.relu(self.batch_nor2(self.conv2(x))))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [33]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
import torch.optim as optim

criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=3e-4, momentum=0.9)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

Epoch = 20

In [ ]:
import matplotlib.pyplot as plt
import torchvision

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(train_loader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(batch_size)))

In [34]:
for epoch in range(Epoch):
    running_loss = 0.0
    running_corrects = 0.0

    val_loss = 0.0
    val_corrects = 0.0
    total = 0

    model.train()
    for idx, data in enumerate(train_loader):
      # get the input; data is a list of [inputs, labels]
      inputs, labels = data
      inputs, labels = inputs.to(device), labels.to(device)

      # zero the parameter gradients
      optimizer.zero_grad()

      # forward + backward + optimize
      outputs = model(inputs)
      loss = criterion(outputs, labels)

      loss.backward()
      optimizer.step()

      # print statistics
      _, preds = torch.max(outputs, 1)
      running_loss += loss.item()
      running_corrects += torch.sum(preds == labels.data)
      if idx % 2000 == 1999:
        print('[Epoch: %d, %5d] loss: %.3f'%(epoch + 1, idx, running_loss / 2000))
        running_loss = 0.0
        running_corrects = 0.0
    
    # -------- Validation --------
    model.eval()
    with torch.no_grad():
      for idx, data in enumerate((val_loader)):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        predict = model(inputs)
        loss = criterion(predict, labels)

        _, vals = torch.max(predict, 1)
        val_loss += loss.item()
        val_corrects += (vals == labels).sum().item()
        total += labels.size(0)
    print('Validation [Epoch: %d, %5d] loss: %.3f'%(epoch + 1, idx, val_loss / total))
    print('Validation Acc: %.3f %%'%(val_corrects / total * 100))

    # -------- Schedule Learning Rate --------
    scheduler.step()
    print("Learning Rate: ", scheduler.get_last_lr()[0])

print('Finish Training')

  0%|          | 0/5236 [00:00<?, ?it/s]


AttributeError: 'list' object has no attribute 'to'

In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname, accuracy))